# CSC_52081_EP Project

Advanced Machine Learning and Autonomous Agents Project

## Introduction

Reinforcement Learning (RL) has emerged as a robust framework for training autonomous agents to learn optimal behaviors through environmental interactions. This study utilizes the [`CarRacing-v3`](https://gymnasium.farama.org/environments/box2d/car_racing/) environment from Gymnasium, which presents a challenging control task in a racing scenario.

### Environment

The environment features a high-dimensional observation space, represented by a $96 \times 96$ RGB image capturing the car and track, necessitating the use of deep convolutional neural networks (CNNs) for effective feature extraction.

#### Action Space

The action space in CarRacing-v3 supports both continuous and discrete control modes.

In **continuous mode**, the agent outputs three real-valued commands:

- steering (ranging from $-1$ to $+1$)
- gas
- braking

In **discrete mode**, the action space is simplified to five actions:

- do nothing
- steer left
- steer right
- gas
- brake

This dual action representation enables a comprehensive evaluation of various RL algorithms under different control settings.

#### Reward

The reward structure combines a penalty of $-0.1$ per frame and a reward of $+\frac{1000}{N}$ for each new track tile visited, where $N$ is the total number of tiles. This incentivizes the agent to balance exploration (visiting tiles) with efficiency (minimizing frame usage). For example, completing the race after visiting all $N$ tiles in 732 frames yields a reward of $1000 - 0.1 \times 732 = 926.8$ points.

### Objective

The primary objective of this project is to compare RL policies across discrete and continuous action modalities. For discrete control, methods like **Deep Q-Network** (DQN) and **SARSA** are implemented, while continuous control is explored using approaches such as the **Cross-Entropy Method** (CEM), **Self-Adaptive Evolution Strategy** (SA-ES), and policy gradient techniques like **Proximal Policy Optimization** (PPO) and **Soft Actor-Critic** (SAC). This comparative analysis aims to understand the strengths and limitations of each method in handling complex decision spaces.

The high-dimensional visual inputs in `CarRacing-v3` require effective feature extraction, addressed through a tailored CNN architecture. Transitioning between discrete and continuous action representations also demands careful algorithmic design and parameter tuning to ensure stable learning and convergence. While prior studies have often focused on either discrete or continuous action spaces separately, this work adopts a comparative approach, evaluating different agents within the same environment to assess performance under similar conditions.

At this stage, the work outlines the methodology and anticipated challenges, focusing on designing the CNN-based feature extractor, implementing RL algorithms, and establishing a framework for performance comparison. Preliminary findings are yet to be finalized, but the study is expected to provide insights into applying RL in high-dimensional, real-time control tasks. Limitations include the preliminary nature of experiments and the need for further tuning and validation. Future work will involve extensive empirical evaluations, exploring additional policy gradient methods, and refining the network architecture to better handle the complexities of `CarRacing-v3`.

### GitHub

The project's code is available on [GitHub](https://github.com/tr0fin0/ensta_CSC_52081_EP_project), offering a reproducible framework for future investigations and extensions.

## Installation

### Environment

#### WSL, Linux or MacOS

A `Python Virtual Environment` will be used for this project by run the following on a terminal on the project folder:

```bash
sudo apt install python3.10-venv
python3 -m venv env
source env/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r requirements.txt
```

### Imports

In [ ]:
from collections import deque
from ipywidgets import interact
from IPython.display import Video
from pathlib import Path
from tqdm.notebook import tqdm
from typing import cast, List, Tuple, Deque, Optional, Callable


import gymnasium as gym
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

In [5]:
sns.set_context("talk")

In [6]:
def video_selector(file_path: List[Path]) -> Video:
    return Video(file_path, embed=True, html_attributes="controls autoplay loop")

### Setup

#### Directories

In [7]:
DIRECTORY_OUTPUT = "output"
DIRECTORY_MODELS = Path(f"{DIRECTORY_OUTPUT}/models/")
DIRECTORY_FIGURES = Path(f"{DIRECTORY_OUTPUT}/images/")

if not DIRECTORY_FIGURES.exists():
    DIRECTORY_FIGURES.mkdir(parents=True)

if not DIRECTORY_MODELS.exists():
    DIRECTORY_MODELS.mkdir(parents=True)

## Demonstration

In [9]:
VIDEO_DEMO = "CSC_52081_EP_demonstration"
(DIRECTORY_FIGURES / f"{VIDEO_DEMO}.mp4").unlink(missing_ok=True)


env = gym.make(
    "CarRacing-v3",
    render_mode="rgb_array",
    lap_complete_percent=0.95,
    domain_randomize=False,
    continuous=False
)
env = gym.wrappers.RecordVideo(env, video_folder=str(DIRECTORY_FIGURES), name_prefix=VIDEO_DEMO)


done = False
observation, info = env.reset()

while not done:
    action = env.action_space.sample()

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()


Video(
    DIRECTORY_FIGURES / f"{VIDEO_DEMO}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)

c:\Users\Mateus\OneDrive\Documentos\7.ENSTA\3A\RI\CSC_52081-AdvancedIA\Project\ensta_CSC_52081_EP_project\env-project\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at c:\Users\Mateus\OneDrive\Documentos\7.ENSTA\3A\RI\CSC_52081-AdvancedIA\Project\ensta_CSC_52081_EP_project\src\figs\videos\project folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


## Description

only demonstration is right. from below here is only experimental.

### Global Definitions

#### Constants

In [ ]:
env = gym.make("CarRacing-v3", render_mode="rgb_array")

STATE_SHAPE = (96, 96, 3)
DISCRETE_ACTIONS = 5
CONTINUOUS_ACTIONS = 3
EPISODES = 1000 

#### Functions

In [ ]:
def plot_learning_curve(rewards):
    plt.plot(rewards)
    plt.xlabel("Episodes")
    plt.ylabel("Cumulative Reward")
    plt.title("Learning Curve")
    plt.show()

def plot_action_heatmap(action_counts):
    plt.imshow(action_counts, cmap="hot", interpolation="nearest")
    plt.colorbar()
    plt.title("Action Heatmap")
    plt.show()

#### CNN Feature Extractor

In [ ]:
import gymnasium.wrappers as gym_wrap
import os

def save(self, save_dir, save_name):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = save_dir + save_name + f"_{self.act_taken}.pt"
    torch.save({
        'upd_model_state_dict': self.updating_net.state_dict(),
        'frz_model_state_dict': self.frozen_net.state_dict(),
        'optimizer_state_dict': self.optimizer.state_dict(),
        # 'replay_buffer': self.buffer,  # Exclude replay buffer from saving
        'action_number': self.act_taken,
        'epsilon': self.epsilon
        }, save_path)
    print(f"Model saved to {save_path} at step {self.act_taken}")

def load(self, load_dir, model_name):
    loaded_model = torch.load(load_dir+model_name, weights_only=False)
    upd_net_param = loaded_model['upd_model_state_dict']
    frz_net_param = loaded_model['frz_model_state_dict']
    opt_param = loaded_model['optimizer_state_dict']
    self.updating_net.load_state_dict(upd_net_param)
    self.frozen_net.load_state_dict(frz_net_param)
    self.optimizer.load_state_dict(opt_param)
    if self.load_state == 'eval':
        self.updating_net.eval()
        self.frozen_net.eval()
        self.epsilon_min = 0
        self.epsilon = 0
    elif self.load_state == 'train':
        self.updating_net.train()
        self.frozen_net.train()
        self.act_taken = loaded_model['action_number']
        self.epsilon = loaded_model['epsilon']
    else:
        raise ValueError(f"Unknown load state. Should be either 'eval' or 'train'.")

class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        for _ in range(self._skip):
            state, reward, terminated, truncated, info = self.env.step(action)
            total_reward += reward
            if terminated:
                break
        return state, total_reward, terminated, truncated, info


class DQN(nn.Module):

    def __init__(self, in_dim, out_dim):
        super().__init__()
        channel_n, height, width = in_dim

        if height != 84 or width != 84:
            raise ValueError(f"DQN model requires input of a (84, 84)-shape. Input of a ({height, width})-shape was passed.")

        self.net = nn.Sequential(
            nn.Conv2d(in_channels=channel_n, out_channels=16,
                      kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32,
                      kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2592, 256),
            nn.ReLU(),
            nn.Linear(256, out_dim),
        )

    def forward(self, input):
        return self.net(input)

env = gym.make(
    "CarRacing-v3",
    continuous=False
)
env = DQN.SkipFrame(env, skip=4)
env = gym_wrap.GrayscaleObservation(env)
env = gym_wrap.ResizeObservation(env, shape=(84, 84))
env = gym_wrap.FrameStackObservation(env, stack_size=4)
state, info = env.reset()
action_n = env.action_space.n

### PPO

In [ ]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Environment setup
env = gym.make("CarRacing-v3")

# Actor-Critic Model Definition
class ActorCritic(tf.keras.Model):
    def __init__(self, action_space):
        super(ActorCritic, self).__init__()
        self.common = layers.Dense(128, activation='relu')
        self.actor = layers.Dense(action_space, activation='tanh')  # For continuous actions
        self.critic = layers.Dense(1)  # Value function output

    def call(self, inputs):
        x = self.common(inputs)
        action_probs = self.actor(x)
        value = self.critic(x)
        return action_probs, value

# Generalized Advantage Estimation (GAE)
def compute_gae(rewards, values, next_values, dones, gamma, lam):
    advantages = []
    gae = 0
    for t in reversed(range(len(rewards))):
        delta = rewards[t] + gamma * next_values[t] * (1 - dones[t]) - values[t]
        gae = delta + gamma * lam * (1 - dones[t]) * gae
        advantages.insert(0, gae)
    return advantages

# Experience Buffer
class ExperienceBuffer:
    def __init__(self, capacity=10000):
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []
        self.values = []
        self.next_states = []

    def add(self, state, action, reward, done, value, next_state):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)
        self.values.append(value)
        self.next_states.append(next_state)

    def clear(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []
        self.values = []
        self.next_states = []

# PPO Update
def ppo_update(model, buffer, optimizer, clip_ratio=0.2, value_loss_coeff=0.5, entropy_coeff=0.01, gamma=0.99, lam=0.95):
    states = np.array(buffer.states)
    actions = np.array(buffer.actions)
    rewards = np.array(buffer.rewards)
    dones = np.array(buffer.dones)
    next_states = np.array(buffer.next_states)
    old_values = np.array(buffer.values)

    # Compute targets
    with tf.GradientTape() as tape:
        action_probs, values = model(states)
        action_probs = action_probs[:, 0]  # Assuming single continuous action
        values = tf.squeeze(values)

        # Calculate advantages
        next_values = model(next_states)[1]
        advantages = compute_gae(rewards, old_values, next_values, dones, gamma, lam)
        advantages = np.array(advantages)

        # Policy Loss (Clipped Objective)
        ratio = tf.exp(action_probs - old_values)
        surrogate_loss = ratio * advantages
        clipped_loss = tf.clip_by_value(ratio, 1 - clip_ratio, 1 + clip_ratio) * advantages
        policy_loss = -tf.reduce_mean(tf.minimum(surrogate_loss, clipped_loss))

        # Value Loss
        value_loss = tf.reduce_mean((values - rewards) ** 2)

        # Entropy Loss
        entropy_loss = -tf.reduce_mean(action_probs * tf.log(action_probs))

        # Total Loss
        loss = policy_loss + value_loss_coeff * value_loss - entropy_coeff * entropy_loss

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

# Training Loop
action_space = env.action_space.shape[0]
model = ActorCritic(action_space)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
buffer = ExperienceBuffer()

for episode in range(1000):
    state = env.reset()
    total_reward = 0

    for t in range(1000):
        state_tensor = tf.convert_to_tensor(state[None, :], dtype=tf.float32)
        action_probs, value = model(state_tensor)
        action = action_probs.numpy().squeeze()
        next_state, reward, done, _ = env.step(action)

        buffer.add(state, action, reward, done, value.numpy(), next_state)

        state = next_state
        total_reward += reward

        if done:
            break

    ppo_update(model, buffer, optimizer)
    buffer.clear()

    print(f"Episode: {episode}, Total Reward: {total_reward}")
